# Thudbot Scaffold - Next part

This will be a prototype Thudbot built in jupyter.

In notebook 00, I went through the steps below to evaluate and choose an appropriate retriever for Thudbot

Steps:
1. ✅ General setup
2. ✅ Data collection and Preparation
3. ✅ SDG with RAGAS to create a golden data set
4. ✅ Setup the RAG chain (finally)
5. ✅ Evaluate results with RAGAS
6. ✅ Refine RAG performance (prompt tuning, retreival methods)

Now I will:
1. Rebuild the final RAG here (without the SDG or RAGAS eval)
2. Add agentic tool calls
3. Add external APIs

Then:
- Convert to a standalone Python script
- Build or reuse a chatbot front end to run it locally


Naming this 01_ 

## Step 1 General setup

In [1]:
### API key management and environment variables

### Reminder: Place .env file inside the root of the project folder so when calling the below from inside the notebook it should find the .env fule and load it inside the notebook environment
### PLEASE ADD THIS `.env` FILE TO YOUR PROJECT'S `.gitignore` file before committing and pushing the changes to your remote repo, as it contains API Keys and Secrets in it

import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env", override=True)

# --- Verify API Keys ---
print("--- API Key Status ---")
print(f"OPENAI_API_KEY loaded: {'OPENAI_API_KEY' in os.environ}")
print(f"LANGCHAIN_API_KEY loaded: {'LANGCHAIN_API_KEY' in os.environ}")
print(f"TAVILY_API_KEY loaded: {'TAVILY_API_KEY' in os.environ}")
print(f"RAGAS_API_KEY loaded: {'RAGAS_API_KEY' in os.environ}")
print(f"ANTHROPIC_API_KEY loaded: {'ANTHROPIC_API_KEY' in os.environ}")
print(f"COHERE_API_KEY loaded: {'COHERE_API_KEY' in os.environ}")
print(f"OPENWEATHER_API_KEY loaded: {'OPENWEATHER_API_KEY' in os.environ}")

# --- Verify General Settings ---
print("\n--- Project Settings Status ---")
print(f"DEBUG mode enabled: {os.environ.get('DEBUG') == 'True'}")
print(f"LangSmith Tracing V2 enabled: {os.environ.get('LANGCHAIN_TRACING_V2') == 'true'}")
print(f"LangChain Project Base: {os.environ.get('LANGCHAIN_PROJECT_BASE')}")
print(f"LangChain Project: {os.environ.get('LANGCHAIN_PROJECT')}")


--- API Key Status ---
OPENAI_API_KEY loaded: True
LANGCHAIN_API_KEY loaded: True
TAVILY_API_KEY loaded: True
RAGAS_API_KEY loaded: False
ANTHROPIC_API_KEY loaded: True
COHERE_API_KEY loaded: True
OPENWEATHER_API_KEY loaded: True

--- Project Settings Status ---
DEBUG mode enabled: True
LangSmith Tracing V2 enabled: True
LangChain Project Base: None
LangChain Project: THUDBOT-CC


including nltk, because it worked before

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/family/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/family/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Step 2: Data Collection and Preparation

My data is CSV structured, so using code from HW9

In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

loader = CSVLoader(
    file_path=f"./data/Thudbot_Hint_Data_1.csv",
    metadata_columns=[
        "question",
        "hint_level",
        "character",
        "speaker",
        "narrative_context",
        "planet",
        "location",
        "category",
        "tone",
        "follow_up_hint_id",
        "answer_keywords",
        "tags"
    ]
)

hint_data = loader.load()

# No need to overwrite page_content; not doing custom transformation
print(hint_data[0].page_content)     # This will already be the hint_text
print(hint_data[0].metadata)         # This will show all the metadata fields


question_id: TSB-001
hint_text: Press the escape key to exit the opening animations
puzzle_name: 
source: self
{'source': './data/Thudbot_Hint_Data_1.csv', 'row': 0, 'question': 'How do I stop the opening movie', 'hint_level': '1', 'character': 'Player', 'speaker': '', 'narrative_context': 'Meta', 'planet': '', 'location': '', 'category': 'Meta', 'tone': '', 'follow_up_hint_id': '', 'answer_keywords': '', 'tags': ''}


### Setting up QDrant! (from HW9)

Now that we have our documents, let's create a QDrant VectorStore with the collection name "ThudbotHints".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

 

In [4]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents=hint_data,
    embedding=embeddings,
    location=":memory:",
    collection_name="Thudbot_Hints"
)

### ▶️ reload platinum data set

Might as well re-use it for any testing

In [5]:
import json

# Load questions for testing retrievers
with open("data/platinum_dataset.json", "r") as f:
    platinum_data = json.load(f)



In [6]:
sample_indices = [0, 2, 5, 7, 10]
sampled_platinum = [platinum_data[i] for i in sample_indices]


## Step 4: Setup the RAG chain


Starting with a "naive" dense vector retrieval

### R - Retrieval - using multi-query retriever, based on previous evaluatin




In [7]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

Moving next cell up in the flow, because multi-query retriever needs LLM

In [8]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")
#chat_model = ChatAnthropic(model="claude-3-5-sonnet-20240620")

In [9]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

### A - Augmented

My first pass at a Thud-like prompt, named as ```THUD_TEMPLATE```

This will need tuning!

In [10]:
from langchain_core.prompts import ChatPromptTemplate

THUD_TEMPLATE = """\
You are Thud, a friendly and somewhat simple-minded patron at The Thirsty Tentacle. 

You're trying your best to help the player navigate the game "The Space Bar."

Use the clues and context provided below to offer a gentle hint — not a full solution.

If you're not sure what to say, admit it honestly or say something silly — like talk about the weather or suggest looking around more.

If the player's question is clearly outside the game's scope (e.g., about real-world topics), 
you may consult the get_weather tool to offer a friendly distraction.

Player's question:
{question}

Context:
{context}

Your hint:"""

rag_prompt = ChatPromptTemplate.from_template(THUD_TEMPLATE)

### G - Generation

Still using `gpt-4.1-nano` as our LLM today

### LCEL RAG Chain

We're going to use LCEL to construct our chain. (from HW9)


Test the chain, and the langsmith tracing with a question.
Might as well take the question from the platinum data set (just remember to load it above ▶️)

Using mq, based on eval results

In [11]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
).with_config({"run_name": "multi_query_chain"})

test with sample questions

In [12]:
# sample_q = platinum_data[0]["eval_sample"]["user_input"]
# sample_q = "What is the best way to get the token?"
sample_q = "why is the weather so nice?"
# sample_q = sampled_platinum[4]["eval_sample"]["user_input"]
multi_query_retrieval_chain.invoke({"question": sample_q})

{'response': AIMessage(content="Hey there, friend! The weather's nice, huh? Sometimes I think the sky just likes to put on a little show. Anyway, since you're wonderin' about the weather, maybe you could take a moment to look around your surroundings. Sometimes little clues hide in plain sight, just like a ray of sunshine peekin' through the tent! Anything else I can help you with?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 3121, 'total_tokens': 3199, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_f12167b370', 'id': 'chatcmpl-C0WVXNUeMzn31DgOLcSEJn847s8wY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--cdf852f8-f8df-487a-946e-264e3371b97a-0', usage_metadata={

use @tool decorator to Wrap multi_query_retrieval_chain as a Tool

In [13]:
from langchain_core.tools import tool

@tool
def hint_lookup(question: str) -> str:
    """Answer in-game player questions or (if needed) give the weather."""
    result = multi_query_retrieval_chain.invoke({"question": question})
    answer = result["response"].content

    if "not sure" in answer.lower() or "look around" in answer.lower():
        return get_weather.invoke({"city": "Boston"})
    
    return answer


let's give thud another tool, using an api call

In [14]:
from langchain_core.tools import tool
import requests
import os

@tool
def get_weather(city: str) -> str:
    """Gets the current weather for a given city using the OpenWeatherMap API."""
    api_key = os.getenv("OPENWEATHER_API_KEY")
    if not api_key:
        return "Oops, no weather API key found."

    try:
        url = (
            f"https://api.openweathermap.org/data/2.5/weather?"
            f"q={city}&units=imperial&appid={api_key}"
        )
        response = requests.get(url)
        data = response.json()

        if response.status_code != 200 or "weather" not in data:
            return f"Couldn't get weather for {city} right now."

        weather = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        return f"It's currently {weather}, around {temp:.0f}°F in {city}."
    
    except Exception as e:
        return f"Weather system error: {e}"


In [15]:
get_weather.invoke("New York")


"It's currently smoke, around 82°F in New York."

In [16]:
from langchain.agents import initialize_agent, AgentType

tools = [hint_lookup]  # just your @tool-wrapped function

thud_agent = initialize_agent(
    tools=tools,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=chat_model,
    verbose=True
)


/var/folders/xc/ddmjsd0x4sl7n58bhfwn6dv00000gn/T/ipykernel_92651/3189070583.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  thud_agent = initialize_agent(


In [17]:
thud_agent.run("How do I get the token from the cup?")
# thud_agent.run("what is the weather?")


/var/folders/xc/ddmjsd0x4sl7n58bhfwn6dv00000gn/T/ipykernel_92651/354438210.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  thud_agent.run("How do I get the token from the cup?")




> Entering new AgentExecutor chain...
Thought: The question suggests the player wants to obtain a token from a cup, likely in a game scenario. I should ask for clarification or details about the game or situation to provide accurate instructions.
Action: hint_lookup
Action Input: How do I get the token from the cup?
Observation: Oh, getting that token out of the cup, huh? Well, I’d say, maybe ask Thud himself to help you snatch it. Sometimes a friendly ask goes a long way! Just make sure you’re holding the cup steady when you do. Looking around for Thud nearby might help — he might be the one to tell you how to take the token without spilling your drink!
Thought:Thought: The hint suggests that either asking Thud directly or waiting for a friendly opportunity to get the token is the way to proceed. Since the advice implies that asking Thud might be the best approach, I will provide that as the guidance.
Final Answer: Try asking Thud directly to help you take the token from the cup. So

'Try asking Thud directly to help you take the token from the cup. Sometimes a friendly request is all it takes!'